# Exploring `plasticparcels` physics kernels in an idealised flow
In this example, we will use `plasticparcels` and an idealised flow field to explore the different physics kernels available.

In [ ]:
# Library imports
from datetime import datetime, timedelta
import xarray as xr
import numpy as np

# parcels and plasticparcels imports
import plasticparcels as pp

# Plotting imports
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import matplotlib.gridspec as gridspec
import matplotlib.ticker as mticker

## Load settings
TODO: UPDATE
We first load in the model settings, and define the simulation settings. For this simulation, we will release the particles at midnight on January 10th 2019. The particle trajectories will be 60 days long, saving the their position every 12 hours. We also set the advection timestep to 20 minutes. By default `plasticparcels` uses 3D advection, so we turn off the 3D-mode, as well as the biofouling behaviour. We ensure that the Stokes drift behaviour and wind-induced drift behaviour is on.

We will also download the necessary release location files (if they are not already downloaded). In our case, as we expect our particles to remain in the Mediterranean Sea, we include `indices` in our ocean model to help speed up the file IO.

In [ ]:
# Create the idealised flow fields
from example_idealised_flow import create_idealised_flow_fields

'''# Load the model settings
settings_file = 'docs/examples/example_idealised_settings.json'
settings = pp.utils.load_settings(settings_file)

# Download the mask and release data
settings = pp.utils.download_plasticparcels_dataset('idealised', settings, 'input_data')
'''

In [ ]:
# Create the simulation settings
settings['simulation'] = {
    'startdate': datetime.strptime('2019-01-01-00:00:00', '%Y-%m-%d-%H:%M:%S'), # Start date of simulation
    'runtime': timedelta(days=30),        # Runtime of simulation
    'outputdt': timedelta(hours=12),      # Timestep of output
    'dt': timedelta(minutes=20),          # Timestep of advection
    }

# Overwrite some settings
settings['use_3D'] = False
settings['use_biofouling'] = False
settings['use_stokes'] = True
settings['use_wind'] = True


In [ ]:
# Create the plastic type settings
settings['plastictype'] = {
    'wind_coefficient' : 0.01,  # Percentage of wind to apply to particles
    'plastic_diameter' : 0.001, # Plastic particle diameter (m)
    'plastic_density' : 1030.,  # Plastic particle density (kg/m^3)
}

## Create a `FieldSet`, `ParticleSet` and `Kernel` list
Here we create the necessary `Parcels` objects to run our simulation. The `FieldSet` contains all the hydrodynamic, wind, and wave data required for our simulation. The `ParticleSet` is a set of particles initialised uniformly over the domain, and the `Kernel` list is a list of kernels that will be applied to these particles. A useful overview of these `Parcels` objects can be found [here](https://docs.oceanparcels.org/en/latest/examples/tutorial_parcels_structure.html).

In [ ]:
# Create the fieldset
fieldset = pp.constructors.create_fieldset(settings)

# Create the particleset
pset = pp.constructors.create_particleset_from_map(fieldset, settings)

# Create the applicable kernels to the plastic particles
kernels = pp.constructors.create_kernel(fieldset)

In [ ]:
runtime = settings['simulation']['runtime']
dt = settings['simulation']['dt']
outputdt = settings['simulation']['outputdt']

## Run the simulation
To run the simulation we create a `ParticleFile` that will store the trajectory information of all particles at the specified output timestep. We then `execute` the simulation with the specified runtime and timestepping.

In [ ]:
# Create the particle file where output will be stored
pfile = pp.ParticleFile('example_idealised.zarr', pset, settings=settings, outputdt=outputdt)

In [ ]:
# Execute the simulation
pset.execute(kernels, runtime=runtime, dt=dt, output_file=pfile)

## Plot the trajectories
Finally, we produce a simple 'spaghetti' plot and and a concentration map of the trajectories to visualise their pathways. To understand how to work with `PlasticParcels` output, please see the `Parcels` tutorial [here](https://docs.oceanparcels.org/en/latest/examples/tutorial_output.html).